## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
import os
from collections import *
%matplotlib inline

## Helper Functions

In [2]:
import math

class RunningMean:
    def __init__(self, max_size = 10, max_prunes = 100):
        self.max_size = max_size
        self.max_prunes = max_prunes
        self.prune_ctr = 0
        self.cache = deque()
        self.sum = 0.0
        
    def __len__(self):
        return len(self.cache)

    def update_mean(self, new_value):
        self.cache.append(new_value)
        self.sum += new_value
        if self.prune():
            self.prune_ctr += 1
            if self.max_prunes > 0 and \
                self.max_prunes < self.prune_ctr:
                self.prune_ctr = 0
                self.recalc()
        return self.get_mean()

    def clear(self):
        self.prune_ctr = 0
        self.cache.clear()
        self.sum = 0.0
    
    def recalc(self):
        self.sum = 0.0
        for val in self.cache:
            self.sum += val

    def prune(self):
        result = False
        while (len(self.cache) > self.max_size):
            result = True
            self.sum -= self.cache.popleft()
        return result

    def get_mean(self):
        return (self.sum/float(len(self.cache)))

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2, 
                extrapolate=False, hide_other=False, hide_vertical=False,
                slope_means = [RunningMean(),RunningMean()],
                inter_means = [RunningMean(),RunningMean()]):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if lines is None:
        return
    
    # Normalized vs standard
    if extrapolate:
        # Get image dimensions
        imshape = img.shape
        ymax = imshape[0]
        xmax = imshape[1]
        xmid = (xmax/2)
        
        ytop = (float(ymax)*0.7)
        ybot = (float(ymax)*1.0)
        
        # Left & right lane line
        line_ctr = [0,0]
        line_weight_sum = [0.0,0.0]
        line_slope_sum = [0.0,0.0]
        line_mid_x_sum = [0.0,0.0]
        line_mid_y_sum = [0.0,0.0]
        
        # Veritcal  lines
        vertical_lines = []
        
        # Normalize
        for line in lines:
            for x1,y1,x2,y2 in line:
                # Vertical lines
                if x2 == x1:
                    vertical_lines.append(line)
                # Other lines
                else:
                    # Calc slope
                    slope = ((y2-y1)/(x2-x1))
                    abs_slope = abs(slope)
                    line_idx = 0 if slope < 0 else 1

                    # Ignore wrong-side lines
                    if abs_slope > 0.5 and \
                        ( ( line_idx == 0 and x1 <= xmid and x2 <= xmid ) or \
                            ( line_idx == 1 and x1 > xmid and x2 > xmid ) ):

                        # Count line
                        line_ctr[line_idx] += 1
                        
                        # Weight line
                        line_len_weight = (math.sqrt(((y2-y1)**2)+((x2-x1)**2))**10)
                        line_weight = (min(10.0, abs_slope)*line_len_weight)
                        line_weight_sum[line_idx] += line_weight
                        line_slope_sum[line_idx] += (slope*line_weight)

                        # X midpoint
                        line_left_x = x1 if x1 < x2 else x2
                        line_mid_x = (line_left_x+(abs(x2-x1)/2))
                        line_mid_x_sum[line_idx] += (line_mid_x*line_weight)
                        
                        # Y midpoint
                        line_top_y = y1 if y1 < y2 else y2
                        line_mid_y = (line_top_y+(abs(y2-y1)/2))
                        line_mid_y_sum[line_idx] += (line_mid_y*line_weight)
        
        # Draw other lines
        if not hide_other:
            # Iterate left/right
            for line_idx in [0, 1]:
                bot_end = None
                top_end = None
                
                # Any lines?
                if line_ctr[line_idx] > 0:
                    # Get mean slope
                    curr_slope = (line_slope_sum[line_idx] / line_weight_sum[line_idx])
                    mean_slope = slope_means[line_idx].update_mean(curr_slope)
                                
                    # Get bottom endpoint
                    curr_mid_x = (line_mid_x_sum[line_idx] / line_weight_sum[line_idx])
                    curr_mid_y = (line_mid_y_sum[line_idx] / line_weight_sum[line_idx])                        
                    curr_inter = ((((curr_mid_y-ybot)/mean_slope)-curr_mid_x)*-1.0)
                    
                    mean_inter = inter_means[line_idx].update_mean(curr_inter)
                    bot_end = [mean_inter,ybot]                    
                    
                    # Get top endpoint
                    top_end = [((((ybot-ytop)/mean_slope)-mean_inter)*-1.0),ytop]

                # Fill in
                elif len(slope_means[line_idx]) > 0 and \
                    len(inter_means[line_idx]) > 0:
                        
                    # Get mean slope
                    mean_slope = slope_means[line_idx].get_mean()
                                
                    # Get bottom endpoint
                    mean_inter = inter_means[line_idx].get_mean()
                    bot_end = [mean_inter,ybot]
                    
                    # Get top endpoint
                    top_end = [((((ybot-ytop)/mean_slope)-mean_inter)*-1.0),ytop]
                        
                # Draw line
                if not bot_end is None and \
                    not top_end is None:
                    cv2.line(img, (int(bot_end[0]), int(bot_end[1])), 
                            (int(top_end[0]), int(top_end[1])), 
                            color, thickness)
                    
        # Draw vertical lines
        if not hide_vertical:
            for line in vertical_lines:
                for x1,y1,x2,y2 in line:
                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                    
    # Not extrapolate
    else:
        # Iterate, draw
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, 
                slope_means = [RunningMean(),RunningMean()],
                inter_means = [RunningMean(),RunningMean()]):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), 
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness=3, color=[255, 0, 0],
                extrapolate=False)
    draw_lines(line_img, lines, thickness=9, color=[0, 0, 255],
                extrapolate=True, 
                slope_means=slope_means, inter_means=inter_means)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [7]:
for curr_image_name in os.listdir('test_images/'):
    # Preliminaries
    print (curr_image_name)
    base_img_name = curr_image_name.replace('.jpg', '')
    png_img_name = (base_img_name + '.png')
    test_dir_name = ('test_images_output/')
    output_dir_name = (test_dir_name + base_img_name + '/')
    if (not os.path.isdir(output_dir_name)):
        os.mkdir(output_dir_name)
    if (not os.path.isdir(test_dir_name + '1_grayscale')):
        os.mkdir(test_dir_name + '1_grayscale')
    if (not os.path.isdir(test_dir_name + '2_gaussian')):
        os.mkdir(test_dir_name + '2_gaussian')
    if (not os.path.isdir(test_dir_name + '3_canny')):
        os.mkdir(test_dir_name + '3_canny')
    if (not os.path.isdir(test_dir_name + '4_masked')):
        os.mkdir(test_dir_name + '4_masked')
    if (not os.path.isdir(test_dir_name + '5_hough')):
        os.mkdir(test_dir_name + '5_hough')
    if (not os.path.isdir(test_dir_name + '6_edges')):
        os.mkdir(test_dir_name + '6_edges')
    if (not os.path.isdir(test_dir_name + '_stacked')):
        os.mkdir(test_dir_name + '_stacked')
    
    # Read in test image  
    image = mpimg.imread('test_images/' + curr_image_name)
    
    # Convert to grayscale
    gray = grayscale(image)
    plt.imsave((output_dir_name + '1_grayscale_' + png_img_name),gray, format="png")
    plt.imsave((test_dir_name + '1_grayscale/1_grayscale_' + png_img_name),gray, format="png")

    # Create gaussian blur version
    kernel_size = 5 # default = 5
    blur_gray = gaussian_blur(gray, kernel_size)
    plt.imsave((output_dir_name + '2_gaussian_' + png_img_name),blur_gray, format="png")
    plt.imsave((test_dir_name + '2_gaussian/2_gaussian_' + png_img_name),blur_gray, format="png")

    # Create Canny edges
    high_threshold = 150 # default = 150
    low_threshold = (high_threshold / 3) # default = 50
    edges = canny(blur_gray, low_threshold, high_threshold)
    plt.imsave((output_dir_name + '3_canny_' + png_img_name),edges, format="png")
    plt.imsave((test_dir_name + '3_canny/3_canny_' + png_img_name),edges, format="png")

    # Create masking
    imshape = image.shape
    ysize = imshape[0]
    xsize = imshape[1]
    vertices = np.array([[(0,ysize),(xsize*0.47, ysize*0.7), \
        (xsize*0.53, ysize*0.7), (xsize,ysize)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    plt.imsave((output_dir_name + '4_masked_' + png_img_name),masked_edges, format="png")
    plt.imsave((test_dir_name + '4_masked/4_masked_' + png_img_name),masked_edges, format="png")

    # Create Hough transform
    rho = 2 # default = 2
    theta = np.pi/180 # default = np.pi/180
    threshold = 15     # default = 15
    min_line_len = 30 # default = 40
    max_line_gap = 20    # default = 20
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
    plt.imsave((output_dir_name + '5_hough_' + png_img_name),line_image, format="png")
    plt.imsave((test_dir_name + '5_hough/5_hough_' + png_img_name),line_image, format="png")

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 
    plt.imsave((output_dir_name + '6_edges_' + png_img_name),color_edges, format="png")
    plt.imsave((test_dir_name + '6_edges/6_edges_' + png_img_name),color_edges, format="png")

    # Draw the lines on the edge image
    lines_edges = weighted_img(image, line_image) 
    plt.imsave((output_dir_name + '_stacked_' + png_img_name),lines_edges, format="png")
    plt.imsave((test_dir_name + '_stacked/_stacked_' + png_img_name),lines_edges, format="png")


solidYellowLeft.jpg
capture-20171114-183155.jpg
solidYellowCurve.jpg
solidYellowCurve2.jpg
capture-20171114-172109.jpg
solidWhiteRight.jpg
whiteCarLaneSwitch.jpg
solidWhiteCurve.jpg


In [4]:
max_cache_size = 10
slope_means = [RunningMean(max_size=max_cache_size),
              RunningMean(max_size=max_cache_size)]
inter_means = [RunningMean(max_size=max_cache_size),
              RunningMean(max_size=max_cache_size)]

def clear_means():
    for slope_mean in slope_means:
        slope_mean.clear()
    for inter_mean in inter_means:
        inter_mean.clear()

def process_image(image):

    # Convert to grayscale
    gray = grayscale(image)

    # Create gaussian blur version
    kernel_size = 5 # default = 5
    blur_gray = gaussian_blur(gray, kernel_size)

    # Create Canny edges
    high_threshold = 150 # default = 150
    low_threshold = (high_threshold / 3) # default = 50
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Create masking
    imshape = image.shape
    ysize = imshape[0]
    xsize = imshape[1]
    vertices = np.array([[(0,(ysize*0.9)),(xsize*0.47, ysize*0.7), \
        (xsize*0.53, ysize*0.7), (xsize,(ysize*0.9))]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    # Create Hough transform
    rho = 2 # default = 2
    theta = (np.pi/180*1.0) # default = np.pi/180
    threshold = 15     # default = 15
    min_line_len = 40 # default = 40
    max_line_gap = 20    # default = 20
    line_image = hough_lines(masked_edges, rho, theta, threshold, 
                            min_line_len, max_line_gap, 
                            slope_means=slope_means, inter_means=inter_means)

    # Draw the lines on the edge image
    return weighted_img(line_image, image, α=0.5)


In [5]:
for input_video_name in os.listdir('test_videos/'):
    # Preliminaries
    input_video_path = ('test_videos/' + input_video_name)
    output_video_path = ('test_videos_output/' + input_video_name)
    print ("input_video_path:", input_video_path)
    print ("output_video_path:", output_video_path)

    clear_means()
    input_video_clip = VideoFileClip(input_video_path)
    output_video_clip = input_video_clip.fl_image(process_image)
    %time output_video_clip.write_videofile(output_video_path, audio=False)
    

input_video_path: test_videos/solidWhiteRight.mp4
output_video_path: test_videos_output/solidWhiteRight.mp4
[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:03<00:00, 73.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.62 s, sys: 444 ms, total: 3.07 s
Wall time: 3.68 s
input_video_path: test_videos/solidYellowLeft.mp4
output_video_path: test_videos_output/solidYellowLeft.mp4
[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:09<00:00, 70.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 8.26 s, sys: 1.04 s, total: 9.31 s
Wall time: 10 s
input_video_path: test_videos/challenge.mp4
output_video_path: test_videos_output/challenge.mp4
[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:06<00:00, 37.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 4.28 s, sys: 888 ms, total: 5.16 s
Wall time: 7.38 s
